# Tiniest Blockchain Example Notebook
Original source found: https://medium.com/crypto-currently/lets-build-the-tiniest-blockchain-e70965a248b



In [24]:
#Import packages
import hashlib as hasher
import pandas
import datetime as date

## SHA Hashing Example
SHA256 hash provides unique alpha numeric code (i.e. "hash") for given value. Able to recreate original value from given hash.

Example below

In [14]:
sha = hasher.sha256()
print sha.hexdigest()

e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855


In [15]:
sha.update("1235asihgasdh")
print sha.hexdigest()

19a02fb23ed4f4d29228b1b2b318cc37f3efdb20859048d3397ff4c57f55dbe6


In [13]:
print hasher.sha256("1235asihgasdh").hexdigest()

19a02fb23ed4f4d29228b1b2b318cc37f3efdb20859048d3397ff4c57f55dbe6


## Creating Blockchain
Blockchain is series of sha or other hashes of data. Hash for given data must also include previous hash.

Provides 'chain' of data history. All hashes contain entire previous history of data; therefore historically immutable when distributed (i.e. stored in multiple locations)

In [23]:
#Each block has timestamp, possible index
#The previous block is used when hasing the current block
#Creates a full history of transactions
# hash0 -> txn1 -> hash(txn1, hash0)-> hash1 -> tnx2 -> hash(tnx2, hash1) -> hash2

class Block:
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.hash_block()
        
    def hash_block(self):
        sha = hasher.sha256()
        sha.update(str(self.index) +
                  str(self.timestamp) + 
                  str(self.data) + 
                  str(self.previous_hash))
        return sha.hexdigest()

### First Block
First transaction constructs first block hash. Function instantiates an object of Block class defined in previous module. Block class object has zero index, arbitrary data of "Gensis Block" and arbitrary hash of  "0". 

In [27]:
def create_genesis_block():
    return Block(0, date.datetime.now(), "Genesis Block", "0")
    

### Next Block
Define rules around instantiated subsequent Block objects. Requires A previous block to instantiate a new block

In [28]:
def next_block(last_block):
    this_index = last_block.index + 1
    this_timestamp = date.datetime.now()
    this_data = "Hey! I'm block " + str(this_index)
    this_previous_hash = last_block.hash
    return Block(this_index, this_timestamp, this_data, this_previous_hash)

## Building the Blockchain

Construct a Blockchain array with sample data. Arry is indexed series of blocks using previous blocks sha hash

In [36]:
## Demonstration of array, not part of application
bc_string = create_genesis_block()
bc_list = [create_genesis_block()]

print bc_string.hash_block()

#has index positions
print bc_list[0].hash_block()


320ce55ac40fde9eb907a8791b2600d25c43d54c5563010712e599f9dc151896
1a704caee428066e2721f36dc410916917126d09c87aa9aee7d95adc83941f70


In [38]:
## actual demostration

blockchain = [create_genesis_block()]
previous_block = blockchain[0]

# number of blocks to create in list

num_of_blocks_to_add = 20 

# adding blocks to python list i.e. "chain"

for i in range(0, num_of_blocks_to_add):
    block_to_add = next_block(previous_block)
    blockchain.append(block_to_add)
    previous_block = block_to_add
    
    print "Block #{} has been added to blockchain!".format(block_to_add.index)
    print "Hash: {}\n".format(block_to_add.hash)



Block #1 has been added to blockchain!
Hash: df18b6f421163f130cc236d4c494ddcf076bff183b5dd2787c6b8c560f6a3f09

Block #2 has been added to blockchain!
Hash: b4cee9ffbcb2f29f59a9616194837ba25c71e9152bf219ddb08641f13a91543e

Block #3 has been added to blockchain!
Hash: a3410d95ef1a626df4daf138291efb89914b81a6eaf17a5b7592b8e3614c768a

Block #4 has been added to blockchain!
Hash: 81d7b0ffdca2de33685606521c4f92bb87792464bbd460e302caf5964bafe75d

Block #5 has been added to blockchain!
Hash: 8604e296adf76dee6e69da66969761ef07bf43224cde5fa9b1d161f3f1a07d81

Block #6 has been added to blockchain!
Hash: 96314990860d37c12f7ce606b97ea543936799cbb16e7b3e8109fbebba54d9e7

Block #7 has been added to blockchain!
Hash: fd965d80fd5d70b4edb5ef92c6ea5335d0848803a7cec4eddecad83b8e04383b

Block #8 has been added to blockchain!
Hash: d541665ecc13ef289ec4df52b596fa8ba27d0ce5d34b074db239777c77cc5ee1

Block #9 has been added to blockchain!
Hash: f8c4fd144d96adcde5d375d55fa356401ddf5a29238abb6b05c70ea7056092ef

B

In [43]:
print blockchain[15].data
print blockchain[15].hash
print blockchain[16].previous_hash

Hey! I'm block 15
b4b521497fce62097d412c01b059cc7ff57c3ecf75d76000339fc9b57b4b57b3
b4b521497fce62097d412c01b059cc7ff57c3ecf75d76000339fc9b57b4b57b3


## Real World Example

Retail store needs to track shipments through complex supply chain. e.g. Walmart fruit goes through many different third parties before arriving in store.

Blockchain gives complete picture of all transactions without requiring all 3rd parties to access central database

In [44]:
def next_block(last_block, data):
    this_index = last_block.index + 1
    this_timestamp = date.datetime.now()
    this_data = data
    this_previous_hash = last_block.hash
    return Block(this_index, this_timestamp, this_data, this_previous_hash)

In [45]:
def shipment(buyer, seller, invoice_id):
    shipment = {"from": seller,
               "to": buyer,
               "invoice_id": invoice_id}
    return shipment

In [75]:
#example
# Python indexes starting with zero
# However, counts starting with one

print len(blockchain)
print blockchain[len(blockchain)-1].hash

1
2f808b560e13abeb22f2d5c668e32f967ee4e079197c44fcf63f776a9799eb33


In [76]:
def get_previous_block(current_blockchain):
    block = current_blockchain[len(current_blockchain)-1]
    return block

In [77]:
def add_shipment_to_blockchain(shipment_info, current_blockchain):
    previous_block = get_previous_block(current_blockchain)
    new_block = next_block(previous_block, shipment_info)
    current_blockchain.append(new_block)
    return current_blockchain

In [88]:
# quality of life function to show data in blockchain

def print_data(current_blockchain):
    for i in current_blockchain:
        print str(i.data) +" " + i.hash

In [90]:
blockchain = [create_genesis_block()]

In [91]:
# Printing data in blockchain
print_data(blockchain)

Genesis Block 622d04d0d7f62d97ff1109e8e2bc605116b391b7ed71106d6f73995b19fc0ecb


In [92]:
# Add transaction to chain
txn1 = shipment("Farm123", "Store234", "1")

In [93]:
blockchain = add_shipment_to_blockchain(txn1, blockchain)

In [94]:
print_data(blockchain)

Genesis Block 622d04d0d7f62d97ff1109e8e2bc605116b391b7ed71106d6f73995b19fc0ecb
{'to': 'Farm123', 'from': 'Store234', 'invoice_id': '1'} e58fc9aa54378a32ba5db895bb5e370304726567f30bf9a2a2f0f1d45c744268


In [96]:
txn2 = shipment("Store234", "Store567", "1")

In [97]:
blockchain = add_shipment_to_blockchain(txn1, blockchain)
print_data(blockchain)

Genesis Block 622d04d0d7f62d97ff1109e8e2bc605116b391b7ed71106d6f73995b19fc0ecb
{'to': 'Farm123', 'from': 'Store234', 'invoice_id': '1'} e58fc9aa54378a32ba5db895bb5e370304726567f30bf9a2a2f0f1d45c744268
{'to': 'Farm123', 'from': 'Store234', 'invoice_id': '1'} 2309e856e85aa0d017f9e0caf732f70899303d0ef5f2f2b4e80676aa11e405a7
